In [1]:
from sympy import *
from sympy.matrices.expressions.matmul import remove_ids
from functools import *
import numpy as np
from scipy.optimize import minimize

In [53]:
A = MatrixSymbol('A',2,2)
_A = Matrix(A)
func = _A[0,0]**2 + _A[1,0]**2

f = lambdify((A),func)

# f: guess -> float
# initial_guess: guess

arr = np.array([[1,2],[3,4]])
f(arr)

ga = np.array([[0, 0], [0, 0]])

def g(a):
    print(type(a), a.shape, a)
    ga[0,:] = a
    return f(ga)
def h(x,y,z,w): return f(np.array([[x,y],[z,w]]))
# print(np.array([0, 0]).shape)

minimize(g, np.array([0, 0]))

<class 'numpy.ndarray'> (2,) [ 0.  0.]
<class 'numpy.ndarray'> (2,) [  1.49011612e-08   0.00000000e+00]
<class 'numpy.ndarray'> (2,) [  0.00000000e+00   1.49011612e-08]
<class 'numpy.ndarray'> (2,) [ 0.  0.]


      fun: 0
 hess_inv: array([[1, 0],
       [0, 1]])
      jac: array([ 0.,  0.])
  message: 'Optimization terminated successfully.'
     nfev: 4
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([ 0.,  0.])

In [103]:
type(arr)

numpy.ndarray

In [94]:
#l = Symbol('l')
l = 3
dim = 2
a = Symbol('a',real = True)
V = MatrixSymbol('V', l, l)
E = MatrixSymbol('E', l, l) # E = [[e1],[e2],[e3],0].T

In [95]:
vE = E * V
fact1 = Q.orthogonal(E)
f = remove_ids(refine(vE.T * vE, fact1))
#f = remove_ids(refine(E[0,:] * E[0,:], facts))
#es = reduce(lambda a, x: a + x**2, [ f[i, j] for i in range(0, dim) for j in range(0, dim)]) - dim

In [96]:
Matrix(f)

Matrix([
[               V[0, 0]**2 + V[1, 0]**2 + V[2, 0]**2, V[0, 0]*V[0, 1] + V[1, 0]*V[1, 1] + V[2, 0]*V[2, 1], V[0, 0]*V[0, 2] + V[1, 0]*V[1, 2] + V[2, 0]*V[2, 2]],
[V[0, 1]*V[0, 0] + V[1, 1]*V[1, 0] + V[2, 1]*V[2, 0],                V[0, 1]**2 + V[1, 1]**2 + V[2, 1]**2, V[0, 1]*V[0, 2] + V[1, 1]*V[1, 2] + V[2, 1]*V[2, 2]],
[V[0, 2]*V[0, 0] + V[1, 2]*V[1, 0] + V[2, 2]*V[2, 0], V[0, 2]*V[0, 1] + V[1, 2]*V[1, 1] + V[2, 2]*V[2, 1],                V[0, 2]**2 + V[1, 2]**2 + V[2, 2]**2]])

In [ ]:
func = sp.Matrix.norm(f)
lam_f = lambdify(var, func, 'numpy')

def lam(x2, y2, p, e_1, e_2):
    return lambda a1,b1,c1,a2,b2,c2,t,s: \
        lam_f(x2, y2, p, e_1, e_2, a1, b1, c1, a2, b2, c2, t, s)

arr_init = np.array([1, 0, 0, 0, 1, 0, 1, 1])

temp1 = E[0]
temp2 = E[1]
x2 = 0
y2 = 0
thisID = 14
f2 = lam(x2, y2, P[thisID].reshape(high_dim, 1), E[0].reshape(high_dim, 1),E[1].reshape(high_dim, 1))
def g(args): return f2(*args)
res = opt.minimize(g, arr_init, method='L-BFGS-B',options={'ftol':1e-3})

In [65]:
B = MatrixSymbol('B',2,2)
C = MatrixSymbol('C',2,2)
_B = Matrix(B)
_C = Matrix(C)

fa = _B[0,1]**2 + _B[0,0]**2 + _C[0,0]**2 + _C[0,1]**2

function = lambdify((B,C),fa)

initial1 = np.array([[1,1],[1,1]])
initial2 = np.array([[1,1],[1,1]])
initial = [initial1, initial2]

def function2(args): return function(*args)

res = opt.minimize(function2, initial, method='L-BFGS-B',options={'ftol':1e-3})

TypeError: <lambda>() takes 2 positional arguments but 8 were given

In [17]:
Es = np.array([[1,0,1,0,1],[0,1,0,1,0]])
P = np.array([3,3,3,3,3])
q = np.array([E[0].dot(P),E[1].dot(P)])
E0 = P - q[0] * E[0] - q[1] * E[1]
E0 = E0 / np.linalg.norm(E0)
E0

array([-0.53452248, -0.26726124, -0.53452248, -0.26726124, -0.53452248])

In [24]:
dim = 2
high_dim = 5

# sympy
A = MatrixSymbol('A',high_dim,high_dim)
q = MatrixSymbol('q', 1, dim)  # values
E = sp.MatrixSymbol('E', high_dim, high_dim) # = (E[0], E[1], E0, P[thisID], 0).T
var = (q,E,A)
_E = E * A  # = (e1' e2' e3' ... R1 R2 ...)

constraints1 = remove_ids(refine(_E.T * _E, Q.orthogonal(E)))

bases_e = Matrix(constraints1[0:dim,0:dim] - Identity(dim))
_bases_e = Matrix.norm(bases_e)

bases_r = Matrix(constraints1[dim:2*dim-1,dim:2*dim-1] - Identity(dim-1))
_bases_r = Matrix.norm(bases_r)

func = _bases_e + _bases_r

lam_f = lambdify(var, func, 'numpy')

In [31]:
def lam(q_, E, P_i):
	E_ = np.zeros(high_dim*high_dim).reshape(high_dim,high_dim)
	partOfE_ = np.r_[E,P_i]
	E_[0:dim+1,0:high_dim] = partOfE_
	return lambda A_: lam_f(q_,E_,A_)

arr_init = np.zeros(high_dim * high_dim).reshape(high_dim, high_dim)

def arr_initializer(a,b):
    arr_init[0:dim+1,0:dim] = a		# E' variables ( E'[0] = this[0:dim+1,0] * (E:E0) )
    arr_init[0:dim,dim:2*dim] = b	# R  variables ( ignore )
    return f2(arr_init)

init_E = np.r_[np.diag([1 for i in range(dim)]),np.zeros(dim).reshape(1,dim)]
init_R = np.ones(dim*(dim-1)).reshape(dim,dim-1)
init = (init_E, init_R)

In [32]:
f2 = lam(np.array([[0,0]]), Es, P.reshape(1,high_dim))
def g(args): return arr_initializer(*args)
g(init)

1.0

In [33]:
res = opt.minimize(g, init, method='L-BFGS-B',options={'ftol':1e-3})

ValueError: setting an array element with a sequence.

In [ ]:
temp1 = Es[0]
temp2 = Es[1]
x2 = 0
y2 = 0
thisID = 14
f2 = lam(np.array([[0,0,0]]), Es, P[14].reshape(1,high_dim))
def g(args): return arr_initializer(*args)
res = opt.minimize(g, init, method='L-BFGS-B',options={'ftol':1e-3})
print(res)